In [ ]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text.RegularExpressions;

var data = File.ReadAllText("inputs/input_day14.txt");

In [ ]:
// Test data
var dataF = 
@"NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C";

In [ ]:
var matcher = new Regex(@"^(\w{2}) -> (\w{1})$", RegexOptions.Compiled);

var template = "";

var map = new Dictionary<string, string[]>();

var pairCounts = new Dictionary<string, long>();

using(var reader = new StringReader(data))
{
    template = reader.ReadLine();

    reader.ReadLine();

    string line;
    while((line = reader.ReadLine()) != null)
    {
        var match = matcher.Match(line);

        if(!match.Success)
        {
            continue;
        }

        var pair = match.Groups[1].Value;
        var insertion = match.Groups[2].Value;

        map.Add(pair, new[]{pair[0] + insertion, insertion + pair[1]});

        pairCounts.Add(pair, 0);
    }
}

Console.WriteLine(template + " " + pairCounts.Count);

KFFNFNNBCNOBCNPFVKCP 100


In [ ]:
for(var index = 0; index < template.Length - 1; ++index)
{
    pairCounts[new string(template.Skip(index).Take(2).ToArray())] += 1;
}

for(var index = 0; index < 40; ++index)
{
    var populated = pairCounts.Where(_ => _.Value > 0);

    var newPairCounts = new Dictionary<string,long>(pairCounts);

    foreach(var pairCount in populated)
    {
        var newPair = map[pairCount.Key];

        newPairCounts[newPair[0]] += pairCount.Value;
        newPairCounts[newPair[1]] += pairCount.Value;

        newPairCounts[pairCount.Key] -= pairCount.Value;
    }

    pairCounts = newPairCounts;
}

record ElementCount(char Element, long Count);

var counts = 
    pairCounts.SelectMany(pair =>
        new []{new ElementCount(pair.Key[0], pair.Value), new ElementCount(pair.Key[1], pair.Value)}
    )
    .Where(_ => _.Count > 0)
    .GroupBy(_ => _.Element)
    .Select(_ => {
        var extra = _.Key == template.First() || _.Key == template.Last() ? 1L : 0L;
        return (_.Select(_ => _.Count).Sum() + extra) / 2L;
    })
    .OrderByDescending(_ => _);

Console.WriteLine(counts.First() - counts.Last());

2158894777814
